# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

This is a modification of the base demonstration notebook provided in the Tensorflow Object Detection module. In addition to any packages you required for the images, you will probably need to instal `skvideo`, either through conda or pip.

# Imports

In [16]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import skvideo.io

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

The same custom paths as before in order to point to the models just trained.

In [4]:
# What model to download.
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './inference_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'fighter_label.pbtxt')

NUM_CLASSES = 1

## Download Model

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [11]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_VIDEOS_DIR = './videos/topgun/'

Below are the paths to multiple videos. Each one has its own cell where it is run through object detection and labeled with bounding boxes. 

In [39]:
# TEST_IMAGE_PATHS = [PATH_TO_TEST_IMAGES_DIR + x for x in os.listdir(PATH_TO_TEST_IMAGES_DIR) if '.jpg' in str.lower(x)]#[ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
VIDEO_PATH =  PATH_TO_VIDEOS_DIR + "videoplayback.3gp"
VIDEO_PATH_HQ = PATH_TO_VIDEOS_DIR + "videoplayback.mp4"
VIDEO_PATH_HQ_2 = PATH_TO_VIDEOS_DIR + "top_gun_dogfight_2.mp4"

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [36]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    ### This is the first bit of custom code. The video we are reading from, and the video
    ### we are writing out to.
    videogen = skvideo.io.vreader(VIDEO_PATH)
    writer = skvideo.io.FFmpegWriter(PATH_TO_VIDEOS_DIR + "outputvideo_skvid.mp4")
    
    i=0
    ### We move through the video frame by frame, reading in a single frame,
    ### running object detection and assigning bounding boxes, and then 
    ### write out this single frame to the video.
    for frame in videogen:
        ### Keep track of which frame we are on/ensure the model is running ###
        print("Frame:",i)
        i += 1
        
        frame_expanded = np.expand_dims(frame, axis=0)

        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        # Draw the results of the detection (aka 'visulaize the results') on the current frame ###
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)
        
        ### Write that frame to the output video ###
        writer.writeFrame(frame)
    writer.close()

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27
Frame: 28
Frame: 29
Frame: 30
Frame: 31
Frame: 32
Frame: 33
Frame: 34
Frame: 35
Frame: 36
Frame: 37
Frame: 38
Frame: 39
Frame: 40
Frame: 41
Frame: 42
Frame: 43
Frame: 44
Frame: 45
Frame: 46
Frame: 47
Frame: 48
Frame: 49
Frame: 50
Frame: 51
Frame: 52
Frame: 53
Frame: 54
Frame: 55
Frame: 56
Frame: 57
Frame: 58
Frame: 59
Frame: 60
Frame: 61
Frame: 62
Frame: 63
Frame: 64
Frame: 65
Frame: 66
Frame: 67
Frame: 68
Frame: 69
Frame: 70
Frame: 71
Frame: 72
Frame: 73
Frame: 74
Frame: 75
Frame: 76
Frame: 77
Frame: 78
Frame: 79
Frame: 80
Frame: 81
Frame: 82
Frame: 83
Frame: 84
Frame: 85
Frame: 86
Frame: 87
Frame: 88
Frame: 89
Frame: 90
Frame: 91
Frame: 92
Frame: 93
Frame: 94
Frame: 95
Frame: 96
Frame: 97
Frame: 98
Frame: 99
Frame: 100

Frame: 757
Frame: 758
Frame: 759
Frame: 760
Frame: 761
Frame: 762
Frame: 763
Frame: 764
Frame: 765
Frame: 766
Frame: 767
Frame: 768
Frame: 769
Frame: 770
Frame: 771
Frame: 772
Frame: 773
Frame: 774
Frame: 775
Frame: 776
Frame: 777
Frame: 778
Frame: 779
Frame: 780
Frame: 781
Frame: 782
Frame: 783
Frame: 784
Frame: 785
Frame: 786
Frame: 787
Frame: 788
Frame: 789
Frame: 790
Frame: 791
Frame: 792
Frame: 793
Frame: 794
Frame: 795
Frame: 796
Frame: 797
Frame: 798
Frame: 799
Frame: 800
Frame: 801
Frame: 802
Frame: 803
Frame: 804
Frame: 805
Frame: 806
Frame: 807
Frame: 808
Frame: 809
Frame: 810
Frame: 811
Frame: 812
Frame: 813
Frame: 814
Frame: 815
Frame: 816
Frame: 817
Frame: 818
Frame: 819
Frame: 820
Frame: 821
Frame: 822
Frame: 823
Frame: 824
Frame: 825
Frame: 826
Frame: 827
Frame: 828
Frame: 829
Frame: 830
Frame: 831
Frame: 832
Frame: 833
Frame: 834
Frame: 835
Frame: 836
Frame: 837
Frame: 838
Frame: 839
Frame: 840
Frame: 841
Frame: 842
Frame: 843
Frame: 844
Frame: 845
Frame: 846
Frame: 847

Frame: 1461
Frame: 1462
Frame: 1463
Frame: 1464
Frame: 1465
Frame: 1466
Frame: 1467
Frame: 1468
Frame: 1469
Frame: 1470
Frame: 1471
Frame: 1472
Frame: 1473
Frame: 1474
Frame: 1475
Frame: 1476
Frame: 1477
Frame: 1478
Frame: 1479
Frame: 1480
Frame: 1481
Frame: 1482
Frame: 1483
Frame: 1484
Frame: 1485
Frame: 1486
Frame: 1487
Frame: 1488
Frame: 1489
Frame: 1490
Frame: 1491
Frame: 1492
Frame: 1493
Frame: 1494
Frame: 1495
Frame: 1496
Frame: 1497
Frame: 1498
Frame: 1499
Frame: 1500
Frame: 1501
Frame: 1502
Frame: 1503
Frame: 1504
Frame: 1505
Frame: 1506
Frame: 1507
Frame: 1508
Frame: 1509
Frame: 1510
Frame: 1511
Frame: 1512
Frame: 1513
Frame: 1514
Frame: 1515
Frame: 1516
Frame: 1517
Frame: 1518
Frame: 1519
Frame: 1520
Frame: 1521
Frame: 1522
Frame: 1523
Frame: 1524
Frame: 1525
Frame: 1526
Frame: 1527
Frame: 1528
Frame: 1529
Frame: 1530
Frame: 1531
Frame: 1532
Frame: 1533
Frame: 1534
Frame: 1535
Frame: 1536
Frame: 1537
Frame: 1538
Frame: 1539
Frame: 1540
Frame: 1541
Frame: 1542
Frame: 1543
Fram

In [38]:
### This is the same as the previous, but just pointing at a separate video file. Please
### see the comments in the above cells.
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    videogen = skvideo.io.vreader(VIDEO_PATH_HQ)
    writer = skvideo.io.FFmpegWriter(PATH_TO_VIDEOS_DIR + "outputvideo_skvid_HQ.mp4")
    
    i=0
    for frame in videogen:
        print("Frame:",i)
        i += 1
        
        frame_expanded = np.expand_dims(frame, axis=0)

        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        # Draw the results of the detection (aka 'visulaize the results')
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)
        
        writer.writeFrame(frame)
    writer.close()

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27
Frame: 28
Frame: 29
Frame: 30
Frame: 31
Frame: 32
Frame: 33
Frame: 34
Frame: 35
Frame: 36
Frame: 37
Frame: 38
Frame: 39
Frame: 40
Frame: 41
Frame: 42
Frame: 43
Frame: 44
Frame: 45
Frame: 46
Frame: 47
Frame: 48
Frame: 49
Frame: 50
Frame: 51
Frame: 52
Frame: 53
Frame: 54
Frame: 55
Frame: 56
Frame: 57
Frame: 58
Frame: 59
Frame: 60
Frame: 61
Frame: 62
Frame: 63
Frame: 64
Frame: 65
Frame: 66
Frame: 67
Frame: 68
Frame: 69
Frame: 70
Frame: 71
Frame: 72
Frame: 73
Frame: 74
Frame: 75
Frame: 76
Frame: 77
Frame: 78
Frame: 79
Frame: 80
Frame: 81
Frame: 82
Frame: 83
Frame: 84
Frame: 85
Frame: 86
Frame: 87
Frame: 88
Frame: 89
Frame: 90
Frame: 91
Frame: 92
Frame: 93
Frame: 94
Frame: 95
Frame: 96
Frame: 97
Frame: 98
Frame: 99
Frame: 100

Frame: 758
Frame: 759
Frame: 760
Frame: 761
Frame: 762
Frame: 763
Frame: 764
Frame: 765
Frame: 766
Frame: 767
Frame: 768
Frame: 769
Frame: 770
Frame: 771
Frame: 772
Frame: 773
Frame: 774
Frame: 775
Frame: 776
Frame: 777
Frame: 778
Frame: 779
Frame: 780
Frame: 781
Frame: 782
Frame: 783
Frame: 784
Frame: 785
Frame: 786
Frame: 787
Frame: 788
Frame: 789
Frame: 790
Frame: 791
Frame: 792
Frame: 793
Frame: 794
Frame: 795
Frame: 796
Frame: 797
Frame: 798
Frame: 799
Frame: 800
Frame: 801
Frame: 802
Frame: 803
Frame: 804
Frame: 805
Frame: 806
Frame: 807
Frame: 808
Frame: 809
Frame: 810
Frame: 811
Frame: 812
Frame: 813
Frame: 814
Frame: 815
Frame: 816
Frame: 817
Frame: 818
Frame: 819
Frame: 820
Frame: 821
Frame: 822
Frame: 823
Frame: 824
Frame: 825
Frame: 826
Frame: 827
Frame: 828
Frame: 829
Frame: 830
Frame: 831
Frame: 832
Frame: 833
Frame: 834
Frame: 835
Frame: 836
Frame: 837
Frame: 838
Frame: 839
Frame: 840
Frame: 841
Frame: 842
Frame: 843
Frame: 844
Frame: 845
Frame: 846
Frame: 847
Frame: 848

Frame: 1461
Frame: 1462
Frame: 1463
Frame: 1464
Frame: 1465
Frame: 1466
Frame: 1467
Frame: 1468
Frame: 1469
Frame: 1470
Frame: 1471
Frame: 1472
Frame: 1473
Frame: 1474
Frame: 1475
Frame: 1476
Frame: 1477
Frame: 1478
Frame: 1479
Frame: 1480
Frame: 1481
Frame: 1482
Frame: 1483
Frame: 1484
Frame: 1485
Frame: 1486
Frame: 1487
Frame: 1488
Frame: 1489
Frame: 1490
Frame: 1491
Frame: 1492
Frame: 1493
Frame: 1494
Frame: 1495
Frame: 1496
Frame: 1497
Frame: 1498
Frame: 1499
Frame: 1500
Frame: 1501
Frame: 1502
Frame: 1503
Frame: 1504
Frame: 1505
Frame: 1506
Frame: 1507
Frame: 1508
Frame: 1509
Frame: 1510
Frame: 1511
Frame: 1512
Frame: 1513
Frame: 1514
Frame: 1515
Frame: 1516
Frame: 1517
Frame: 1518
Frame: 1519
Frame: 1520
Frame: 1521
Frame: 1522
Frame: 1523
Frame: 1524
Frame: 1525
Frame: 1526
Frame: 1527
Frame: 1528
Frame: 1529
Frame: 1530
Frame: 1531
Frame: 1532
Frame: 1533
Frame: 1534
Frame: 1535
Frame: 1536
Frame: 1537
Frame: 1538
Frame: 1539
Frame: 1540
Frame: 1541
Frame: 1542
Frame: 1543
Fram

Frame: 2144
Frame: 2145
Frame: 2146
Frame: 2147
Frame: 2148
Frame: 2149
Frame: 2150
Frame: 2151
Frame: 2152
Frame: 2153
Frame: 2154
Frame: 2155
Frame: 2156
Frame: 2157
Frame: 2158
Frame: 2159
Frame: 2160
Frame: 2161
Frame: 2162
Frame: 2163
Frame: 2164
Frame: 2165
Frame: 2166
Frame: 2167
Frame: 2168
Frame: 2169
Frame: 2170
Frame: 2171
Frame: 2172
Frame: 2173
Frame: 2174
Frame: 2175
Frame: 2176
Frame: 2177
Frame: 2178
Frame: 2179
Frame: 2180
Frame: 2181
Frame: 2182
Frame: 2183
Frame: 2184
Frame: 2185
Frame: 2186
Frame: 2187
Frame: 2188
Frame: 2189
Frame: 2190
Frame: 2191
Frame: 2192
Frame: 2193
Frame: 2194
Frame: 2195
Frame: 2196
Frame: 2197
Frame: 2198
Frame: 2199
Frame: 2200
Frame: 2201
Frame: 2202
Frame: 2203
Frame: 2204
Frame: 2205
Frame: 2206
Frame: 2207
Frame: 2208
Frame: 2209
Frame: 2210
Frame: 2211
Frame: 2212
Frame: 2213
Frame: 2214
Frame: 2215
Frame: 2216
Frame: 2217
Frame: 2218
Frame: 2219
Frame: 2220
Frame: 2221
Frame: 2222
Frame: 2223
Frame: 2224
Frame: 2225
Frame: 2226
Fram

Frame: 2828
Frame: 2829
Frame: 2830
Frame: 2831
Frame: 2832
Frame: 2833
Frame: 2834
Frame: 2835
Frame: 2836
Frame: 2837
Frame: 2838
Frame: 2839
Frame: 2840
Frame: 2841
Frame: 2842
Frame: 2843
Frame: 2844
Frame: 2845
Frame: 2846
Frame: 2847
Frame: 2848
Frame: 2849
Frame: 2850
Frame: 2851
Frame: 2852
Frame: 2853
Frame: 2854
Frame: 2855
Frame: 2856
Frame: 2857
Frame: 2858
Frame: 2859
Frame: 2860
Frame: 2861
Frame: 2862
Frame: 2863
Frame: 2864
Frame: 2865
Frame: 2866
Frame: 2867
Frame: 2868
Frame: 2869
Frame: 2870
Frame: 2871
Frame: 2872
Frame: 2873
Frame: 2874
Frame: 2875
Frame: 2876
Frame: 2877
Frame: 2878
Frame: 2879
Frame: 2880
Frame: 2881
Frame: 2882
Frame: 2883
Frame: 2884
Frame: 2885
Frame: 2886
Frame: 2887
Frame: 2888
Frame: 2889
Frame: 2890
Frame: 2891
Frame: 2892
Frame: 2893
Frame: 2894
Frame: 2895
Frame: 2896
Frame: 2897
Frame: 2898
Frame: 2899
Frame: 2900
Frame: 2901
Frame: 2902
Frame: 2903
Frame: 2904
Frame: 2905
Frame: 2906
Frame: 2907
Frame: 2908
Frame: 2909
Frame: 2910
Fram

Frame: 3512
Frame: 3513
Frame: 3514
Frame: 3515
Frame: 3516
Frame: 3517
Frame: 3518
Frame: 3519
Frame: 3520
Frame: 3521
Frame: 3522
Frame: 3523
Frame: 3524
Frame: 3525
Frame: 3526
Frame: 3527
Frame: 3528
Frame: 3529
Frame: 3530
Frame: 3531
Frame: 3532
Frame: 3533
Frame: 3534
Frame: 3535
Frame: 3536
Frame: 3537
Frame: 3538
Frame: 3539
Frame: 3540
Frame: 3541
Frame: 3542
Frame: 3543
Frame: 3544
Frame: 3545
Frame: 3546
Frame: 3547
Frame: 3548
Frame: 3549
Frame: 3550
Frame: 3551
Frame: 3552
Frame: 3553
Frame: 3554
Frame: 3555
Frame: 3556
Frame: 3557
Frame: 3558
Frame: 3559
Frame: 3560
Frame: 3561
Frame: 3562
Frame: 3563
Frame: 3564
Frame: 3565
Frame: 3566
Frame: 3567
Frame: 3568
Frame: 3569
Frame: 3570
Frame: 3571
Frame: 3572
Frame: 3573
Frame: 3574
Frame: 3575
Frame: 3576
Frame: 3577
Frame: 3578
Frame: 3579
Frame: 3580
Frame: 3581
Frame: 3582
Frame: 3583
Frame: 3584
Frame: 3585
Frame: 3586
Frame: 3587
Frame: 3588
Frame: 3589
Frame: 3590
Frame: 3591
Frame: 3592
Frame: 3593
Frame: 3594
Fram

In [40]:
### This is the same as the previous, but just pointing at a separate video file. Please
### see the comments in the above cells.
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    videogen = skvideo.io.vreader(VIDEO_PATH_HQ_2)
    writer = skvideo.io.FFmpegWriter(PATH_TO_VIDEOS_DIR + "top_gun_skvid_HQ_2.mp4")
    
    i=0
    for frame in videogen:
        print("Frame:",i)
        i += 1
        
        frame_expanded = np.expand_dims(frame, axis=0)

        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        # Draw the results of the detection (aka 'visulaize the results')
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)
        
        writer.writeFrame(frame)
    writer.close()

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27
Frame: 28
Frame: 29
Frame: 30
Frame: 31
Frame: 32
Frame: 33
Frame: 34
Frame: 35
Frame: 36
Frame: 37
Frame: 38
Frame: 39
Frame: 40
Frame: 41
Frame: 42
Frame: 43
Frame: 44
Frame: 45
Frame: 46
Frame: 47
Frame: 48
Frame: 49
Frame: 50
Frame: 51
Frame: 52
Frame: 53
Frame: 54
Frame: 55
Frame: 56
Frame: 57
Frame: 58
Frame: 59
Frame: 60
Frame: 61
Frame: 62
Frame: 63
Frame: 64
Frame: 65
Frame: 66
Frame: 67
Frame: 68
Frame: 69
Frame: 70
Frame: 71
Frame: 72
Frame: 73
Frame: 74
Frame: 75
Frame: 76
Frame: 77
Frame: 78
Frame: 79
Frame: 80
Frame: 81
Frame: 82
Frame: 83
Frame: 84
Frame: 85
Frame: 86
Frame: 87
Frame: 88
Frame: 89
Frame: 90
Frame: 91
Frame: 92
Frame: 93
Frame: 94
Frame: 95
Frame: 96
Frame: 97
Frame: 98
Frame: 99
Frame: 100

Frame: 756
Frame: 757
Frame: 758
Frame: 759
Frame: 760
Frame: 761
Frame: 762
Frame: 763
Frame: 764
Frame: 765
Frame: 766
Frame: 767
Frame: 768
Frame: 769
Frame: 770
Frame: 771
Frame: 772
Frame: 773
Frame: 774
Frame: 775
Frame: 776
Frame: 777
Frame: 778
Frame: 779
Frame: 780
Frame: 781
Frame: 782
Frame: 783
Frame: 784
Frame: 785
Frame: 786
Frame: 787
Frame: 788
Frame: 789
Frame: 790
Frame: 791
Frame: 792
Frame: 793
Frame: 794
Frame: 795
Frame: 796
Frame: 797
Frame: 798
Frame: 799
Frame: 800
Frame: 801
Frame: 802
Frame: 803
Frame: 804
Frame: 805
Frame: 806
Frame: 807
Frame: 808
Frame: 809
Frame: 810
Frame: 811
Frame: 812
Frame: 813
Frame: 814
Frame: 815
Frame: 816
Frame: 817
Frame: 818
Frame: 819
Frame: 820
Frame: 821
Frame: 822
Frame: 823
Frame: 824
Frame: 825
Frame: 826
Frame: 827
Frame: 828
Frame: 829
Frame: 830
Frame: 831
Frame: 832
Frame: 833
Frame: 834
Frame: 835
Frame: 836
Frame: 837
Frame: 838
Frame: 839
Frame: 840
Frame: 841
Frame: 842
Frame: 843
Frame: 844
Frame: 845
Frame: 846

Frame: 1462
Frame: 1463
Frame: 1464
Frame: 1465
Frame: 1466
Frame: 1467
Frame: 1468
Frame: 1469
Frame: 1470
Frame: 1471
Frame: 1472
Frame: 1473
Frame: 1474
Frame: 1475
Frame: 1476
Frame: 1477
Frame: 1478
Frame: 1479
Frame: 1480
Frame: 1481
Frame: 1482
Frame: 1483
Frame: 1484
Frame: 1485
Frame: 1486
Frame: 1487
Frame: 1488
Frame: 1489
Frame: 1490
Frame: 1491
Frame: 1492
Frame: 1493
Frame: 1494
Frame: 1495
Frame: 1496
Frame: 1497
Frame: 1498
Frame: 1499
Frame: 1500
Frame: 1501
Frame: 1502
Frame: 1503
Frame: 1504
Frame: 1505
Frame: 1506
Frame: 1507
Frame: 1508
Frame: 1509
Frame: 1510
Frame: 1511
Frame: 1512
Frame: 1513
Frame: 1514
Frame: 1515
Frame: 1516
Frame: 1517
Frame: 1518
Frame: 1519
Frame: 1520
Frame: 1521
Frame: 1522
Frame: 1523
Frame: 1524
Frame: 1525
Frame: 1526
Frame: 1527
Frame: 1528
Frame: 1529
Frame: 1530
Frame: 1531
Frame: 1532
Frame: 1533
Frame: 1534
Frame: 1535
Frame: 1536
Frame: 1537
Frame: 1538
Frame: 1539
Frame: 1540
Frame: 1541
Frame: 1542
Frame: 1543
Frame: 1544
Fram

Frame: 2145
Frame: 2146
Frame: 2147
Frame: 2148
Frame: 2149
Frame: 2150
Frame: 2151
Frame: 2152
Frame: 2153
Frame: 2154
Frame: 2155
Frame: 2156
Frame: 2157
Frame: 2158
Frame: 2159
Frame: 2160
Frame: 2161
Frame: 2162
Frame: 2163
Frame: 2164
Frame: 2165
Frame: 2166
Frame: 2167
Frame: 2168
Frame: 2169
Frame: 2170
Frame: 2171
Frame: 2172
Frame: 2173
Frame: 2174
Frame: 2175
Frame: 2176
Frame: 2177
Frame: 2178
Frame: 2179
Frame: 2180
Frame: 2181
Frame: 2182
Frame: 2183
Frame: 2184
Frame: 2185
Frame: 2186
Frame: 2187
Frame: 2188
Frame: 2189
Frame: 2190
Frame: 2191
Frame: 2192
Frame: 2193
Frame: 2194
Frame: 2195
Frame: 2196
Frame: 2197
Frame: 2198
Frame: 2199
Frame: 2200
Frame: 2201
Frame: 2202
Frame: 2203
Frame: 2204
Frame: 2205
Frame: 2206
Frame: 2207
Frame: 2208
Frame: 2209
Frame: 2210
Frame: 2211
Frame: 2212
Frame: 2213
Frame: 2214
Frame: 2215
Frame: 2216
Frame: 2217
Frame: 2218
Frame: 2219
Frame: 2220
Frame: 2221
Frame: 2222
Frame: 2223
Frame: 2224
Frame: 2225
Frame: 2226
Frame: 2227
Fram

Frame: 2829
Frame: 2830
Frame: 2831
Frame: 2832
Frame: 2833
Frame: 2834
Frame: 2835
Frame: 2836
Frame: 2837
Frame: 2838
Frame: 2839
Frame: 2840
Frame: 2841
Frame: 2842
Frame: 2843
Frame: 2844
Frame: 2845
Frame: 2846
Frame: 2847
Frame: 2848
Frame: 2849
Frame: 2850
Frame: 2851
Frame: 2852
Frame: 2853
Frame: 2854
Frame: 2855
Frame: 2856
Frame: 2857
Frame: 2858
Frame: 2859
Frame: 2860
Frame: 2861
Frame: 2862
Frame: 2863
Frame: 2864
Frame: 2865
Frame: 2866
Frame: 2867
Frame: 2868
Frame: 2869
Frame: 2870
Frame: 2871
Frame: 2872
Frame: 2873
Frame: 2874
Frame: 2875
Frame: 2876
Frame: 2877
Frame: 2878
Frame: 2879
Frame: 2880
Frame: 2881
Frame: 2882
Frame: 2883
Frame: 2884
Frame: 2885
Frame: 2886
Frame: 2887
Frame: 2888
Frame: 2889
Frame: 2890
Frame: 2891
Frame: 2892
Frame: 2893
Frame: 2894
Frame: 2895
Frame: 2896
Frame: 2897
Frame: 2898
Frame: 2899
Frame: 2900
Frame: 2901
Frame: 2902
Frame: 2903
Frame: 2904
Frame: 2905
Frame: 2906
Frame: 2907
Frame: 2908
Frame: 2909
Frame: 2910
Frame: 2911
Fram

Frame: 3515
Frame: 3516
Frame: 3517
Frame: 3518
Frame: 3519
Frame: 3520
Frame: 3521
Frame: 3522
Frame: 3523
Frame: 3524
Frame: 3525
Frame: 3526
Frame: 3527
Frame: 3528
Frame: 3529
Frame: 3530
Frame: 3531
Frame: 3532
Frame: 3533
Frame: 3534
Frame: 3535
Frame: 3536
Frame: 3537
Frame: 3538
Frame: 3539
Frame: 3540
Frame: 3541
Frame: 3542
Frame: 3543
Frame: 3544
Frame: 3545
Frame: 3546
Frame: 3547
Frame: 3548
Frame: 3549
Frame: 3550
Frame: 3551
Frame: 3552
Frame: 3553
Frame: 3554
Frame: 3555
Frame: 3556
Frame: 3557
Frame: 3558
Frame: 3559
Frame: 3560
Frame: 3561
Frame: 3562
Frame: 3563
Frame: 3564
Frame: 3565
Frame: 3566
Frame: 3567
Frame: 3568
Frame: 3569
Frame: 3570
Frame: 3571
Frame: 3572
Frame: 3573
Frame: 3574
Frame: 3575
Frame: 3576
Frame: 3577
Frame: 3578
Frame: 3579
Frame: 3580
Frame: 3581
Frame: 3582
Frame: 3583
Frame: 3584
Frame: 3585
Frame: 3586
Frame: 3587
Frame: 3588
Frame: 3589
Frame: 3590
Frame: 3591
Frame: 3592
Frame: 3593
Frame: 3594
Frame: 3595
Frame: 3596
Frame: 3597
Fram

In [41]:
### This is the same as the previous, but just pointing at a separate video file. Please
### see the comments in the above cells.
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    videogen = skvideo.io.vreader(PATH_TO_VIDEOS_DIR + "f22.mp4")
    writer = skvideo.io.FFmpegWriter(PATH_TO_VIDEOS_DIR + "f22_skvid_HQ.mp4")
    
    i=0
    for frame in videogen:
        print("Frame:",i)
        i += 1
        
        frame_expanded = np.expand_dims(frame, axis=0)

        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        # Draw the results of the detection (aka 'visulaize the results')
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)
        
        writer.writeFrame(frame)
    writer.close()

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27
Frame: 28
Frame: 29
Frame: 30
Frame: 31
Frame: 32
Frame: 33
Frame: 34
Frame: 35
Frame: 36
Frame: 37
Frame: 38
Frame: 39
Frame: 40
Frame: 41
Frame: 42
Frame: 43
Frame: 44
Frame: 45
Frame: 46
Frame: 47
Frame: 48
Frame: 49
Frame: 50
Frame: 51
Frame: 52
Frame: 53
Frame: 54
Frame: 55
Frame: 56
Frame: 57
Frame: 58
Frame: 59
Frame: 60
Frame: 61
Frame: 62
Frame: 63
Frame: 64
Frame: 65
Frame: 66
Frame: 67
Frame: 68
Frame: 69
Frame: 70
Frame: 71
Frame: 72
Frame: 73
Frame: 74
Frame: 75
Frame: 76
Frame: 77
Frame: 78
Frame: 79
Frame: 80
Frame: 81
Frame: 82
Frame: 83
Frame: 84
Frame: 85
Frame: 86
Frame: 87
Frame: 88
Frame: 89
Frame: 90
Frame: 91
Frame: 92
Frame: 93
Frame: 94
Frame: 95
Frame: 96
Frame: 97
Frame: 98
Frame: 99
Frame: 100

Frame: 757
Frame: 758
Frame: 759
Frame: 760
Frame: 761
Frame: 762
Frame: 763
Frame: 764
Frame: 765
Frame: 766
Frame: 767
Frame: 768
Frame: 769
Frame: 770
Frame: 771
Frame: 772
Frame: 773
Frame: 774
Frame: 775
Frame: 776
Frame: 777
Frame: 778
Frame: 779
Frame: 780
Frame: 781
Frame: 782
Frame: 783
Frame: 784
Frame: 785
Frame: 786
Frame: 787
Frame: 788
Frame: 789
Frame: 790
Frame: 791
Frame: 792
Frame: 793
Frame: 794
Frame: 795
Frame: 796
Frame: 797
Frame: 798
Frame: 799
Frame: 800
Frame: 801
Frame: 802
Frame: 803
Frame: 804
Frame: 805
Frame: 806
Frame: 807
Frame: 808
Frame: 809
Frame: 810
Frame: 811
Frame: 812
Frame: 813
Frame: 814
Frame: 815
Frame: 816
Frame: 817
Frame: 818
Frame: 819
Frame: 820
Frame: 821
Frame: 822
Frame: 823
Frame: 824
Frame: 825
Frame: 826
Frame: 827
Frame: 828
Frame: 829
Frame: 830
Frame: 831
Frame: 832
Frame: 833
Frame: 834
Frame: 835
Frame: 836
Frame: 837
Frame: 838
Frame: 839
Frame: 840
Frame: 841
Frame: 842
Frame: 843
Frame: 844
Frame: 845
Frame: 846
Frame: 847

Frame: 1461
Frame: 1462
Frame: 1463
Frame: 1464
Frame: 1465
Frame: 1466
Frame: 1467
Frame: 1468
Frame: 1469
Frame: 1470
Frame: 1471
Frame: 1472
Frame: 1473
Frame: 1474
Frame: 1475
Frame: 1476
Frame: 1477
Frame: 1478
Frame: 1479
Frame: 1480
Frame: 1481
Frame: 1482
Frame: 1483
Frame: 1484
Frame: 1485
Frame: 1486
Frame: 1487
Frame: 1488
Frame: 1489
Frame: 1490
Frame: 1491
Frame: 1492
Frame: 1493
Frame: 1494
Frame: 1495
Frame: 1496
Frame: 1497
Frame: 1498
Frame: 1499
Frame: 1500
Frame: 1501
Frame: 1502
Frame: 1503
Frame: 1504
Frame: 1505
Frame: 1506
Frame: 1507
Frame: 1508
Frame: 1509
Frame: 1510
Frame: 1511
Frame: 1512
Frame: 1513
Frame: 1514
Frame: 1515
Frame: 1516
Frame: 1517
Frame: 1518
Frame: 1519
Frame: 1520
Frame: 1521
Frame: 1522
Frame: 1523
Frame: 1524
Frame: 1525
Frame: 1526
Frame: 1527
Frame: 1528
Frame: 1529
Frame: 1530
Frame: 1531
Frame: 1532
Frame: 1533
Frame: 1534
Frame: 1535
Frame: 1536
Frame: 1537
Frame: 1538
Frame: 1539
Frame: 1540
Frame: 1541
Frame: 1542
Frame: 1543
Fram

Frame: 2144
Frame: 2145
Frame: 2146
Frame: 2147
Frame: 2148
Frame: 2149
Frame: 2150
Frame: 2151
Frame: 2152
Frame: 2153
Frame: 2154
Frame: 2155
Frame: 2156
Frame: 2157
Frame: 2158
Frame: 2159
Frame: 2160
Frame: 2161
Frame: 2162
Frame: 2163
Frame: 2164
Frame: 2165
Frame: 2166
Frame: 2167
Frame: 2168
Frame: 2169
Frame: 2170
Frame: 2171
Frame: 2172
Frame: 2173
Frame: 2174
Frame: 2175
Frame: 2176
Frame: 2177
Frame: 2178
Frame: 2179
Frame: 2180
Frame: 2181
Frame: 2182
Frame: 2183
Frame: 2184
Frame: 2185
Frame: 2186
Frame: 2187
Frame: 2188
Frame: 2189
Frame: 2190
Frame: 2191
Frame: 2192
Frame: 2193
Frame: 2194
Frame: 2195
Frame: 2196
Frame: 2197
Frame: 2198
Frame: 2199
Frame: 2200
Frame: 2201
Frame: 2202
Frame: 2203
Frame: 2204
Frame: 2205
Frame: 2206
Frame: 2207
Frame: 2208
Frame: 2209
Frame: 2210
Frame: 2211
Frame: 2212
Frame: 2213
Frame: 2214
Frame: 2215
Frame: 2216
Frame: 2217
Frame: 2218
Frame: 2219
Frame: 2220
Frame: 2221
Frame: 2222
Frame: 2223
Frame: 2224
Frame: 2225
Frame: 2226
Fram

Frame: 2828
Frame: 2829
Frame: 2830
Frame: 2831
Frame: 2832
Frame: 2833
Frame: 2834
Frame: 2835
Frame: 2836
Frame: 2837
Frame: 2838
Frame: 2839
Frame: 2840
Frame: 2841
Frame: 2842
Frame: 2843
Frame: 2844
Frame: 2845
Frame: 2846
Frame: 2847
Frame: 2848
Frame: 2849
Frame: 2850
Frame: 2851
Frame: 2852
Frame: 2853
Frame: 2854
Frame: 2855
Frame: 2856
Frame: 2857
Frame: 2858
Frame: 2859
Frame: 2860
Frame: 2861
Frame: 2862
Frame: 2863
Frame: 2864
Frame: 2865
Frame: 2866
Frame: 2867
Frame: 2868
Frame: 2869
Frame: 2870
Frame: 2871
Frame: 2872
Frame: 2873
Frame: 2874
Frame: 2875
Frame: 2876
Frame: 2877
Frame: 2878
Frame: 2879
Frame: 2880
Frame: 2881
Frame: 2882
Frame: 2883
Frame: 2884
Frame: 2885
Frame: 2886
Frame: 2887
Frame: 2888
Frame: 2889
Frame: 2890
Frame: 2891
Frame: 2892
Frame: 2893
Frame: 2894
Frame: 2895
Frame: 2896
Frame: 2897
Frame: 2898
Frame: 2899
Frame: 2900
Frame: 2901
Frame: 2902
Frame: 2903
Frame: 2904
Frame: 2905
Frame: 2906
Frame: 2907
Frame: 2908
Frame: 2909
Frame: 2910
Fram

...

Frame: 8989
Frame: 8990
Frame: 8991
Frame: 8992
Frame: 8993
Frame: 8994
Frame: 8995
Frame: 8996
Frame: 8997
Frame: 8998
Frame: 8999
Frame: 9000
Frame: 9001
Frame: 9002
Frame: 9003
Frame: 9004
Frame: 9005
Frame: 9006
Frame: 9007
Frame: 9008
Frame: 9009
Frame: 9010
Frame: 9011
Frame: 9012
Frame: 9013
Frame: 9014
Frame: 9015
Frame: 9016
Frame: 9017
Frame: 9018
Frame: 9019
Frame: 9020
Frame: 9021
Frame: 9022
Frame: 9023
Frame: 9024
Frame: 9025
Frame: 9026
Frame: 9027
Frame: 9028
Frame: 9029
Frame: 9030
Frame: 9031
Frame: 9032
Frame: 9033
Frame: 9034
Frame: 9035
Frame: 9036
Frame: 9037
Frame: 9038
Frame: 9039
Frame: 9040
Frame: 9041
Frame: 9042
Frame: 9043
Frame: 9044
Frame: 9045
Frame: 9046
Frame: 9047
Frame: 9048
Frame: 9049
Frame: 9050
Frame: 9051
Frame: 9052
Frame: 9053
Frame: 9054
Frame: 9055
Frame: 9056
Frame: 9057
Frame: 9058
Frame: 9059
Frame: 9060
Frame: 9061
Frame: 9062
Frame: 9063
Frame: 9064
Frame: 9065
Frame: 9066
Frame: 9067
Frame: 9068
Frame: 9069
Frame: 9070
Frame: 9071
Fram

Frame: 9674
Frame: 9675
Frame: 9676
Frame: 9677
Frame: 9678
Frame: 9679
Frame: 9680
Frame: 9681
Frame: 9682
Frame: 9683
Frame: 9684
Frame: 9685
Frame: 9686
Frame: 9687
Frame: 9688
Frame: 9689
Frame: 9690
Frame: 9691
Frame: 9692
Frame: 9693
Frame: 9694
Frame: 9695
Frame: 9696
Frame: 9697
Frame: 9698
Frame: 9699
Frame: 9700
Frame: 9701
Frame: 9702
Frame: 9703
Frame: 9704
Frame: 9705
Frame: 9706
Frame: 9707
Frame: 9708
Frame: 9709
Frame: 9710
Frame: 9711
Frame: 9712
Frame: 9713
Frame: 9714
Frame: 9715
Frame: 9716
Frame: 9717
Frame: 9718
Frame: 9719
Frame: 9720
Frame: 9721
Frame: 9722
Frame: 9723
Frame: 9724
Frame: 9725
Frame: 9726
Frame: 9727
Frame: 9728
Frame: 9729
Frame: 9730
Frame: 9731
Frame: 9732
Frame: 9733
Frame: 9734
Frame: 9735
Frame: 9736
Frame: 9737
Frame: 9738
Frame: 9739
Frame: 9740
Frame: 9741
Frame: 9742
Frame: 9743
Frame: 9744
Frame: 9745
Frame: 9746
Frame: 9747
Frame: 9748
Frame: 9749
Frame: 9750
Frame: 9751
Frame: 9752
Frame: 9753
Frame: 9754
Frame: 9755
Frame: 9756
Fram

Frame: 10332
Frame: 10333
Frame: 10334
Frame: 10335
Frame: 10336
Frame: 10337
Frame: 10338
Frame: 10339
Frame: 10340
Frame: 10341
Frame: 10342
Frame: 10343
Frame: 10344
Frame: 10345
Frame: 10346
Frame: 10347
Frame: 10348
Frame: 10349
Frame: 10350
Frame: 10351
Frame: 10352
Frame: 10353
Frame: 10354
Frame: 10355
Frame: 10356
Frame: 10357
Frame: 10358
Frame: 10359
Frame: 10360
Frame: 10361
Frame: 10362
Frame: 10363
Frame: 10364
Frame: 10365
Frame: 10366
Frame: 10367
Frame: 10368
Frame: 10369
Frame: 10370
Frame: 10371
Frame: 10372
Frame: 10373
Frame: 10374
Frame: 10375
Frame: 10376
Frame: 10377
Frame: 10378
Frame: 10379
Frame: 10380
Frame: 10381
Frame: 10382
Frame: 10383
Frame: 10384
Frame: 10385
Frame: 10386
Frame: 10387
Frame: 10388
Frame: 10389
Frame: 10390
Frame: 10391
Frame: 10392
Frame: 10393
Frame: 10394
Frame: 10395
Frame: 10396
Frame: 10397
Frame: 10398
Frame: 10399
Frame: 10400
Frame: 10401
Frame: 10402
Frame: 10403
Frame: 10404
Frame: 10405
Frame: 10406
Frame: 10407
Frame: 10408

Frame: 10963
Frame: 10964
Frame: 10965
Frame: 10966
Frame: 10967
Frame: 10968
Frame: 10969
Frame: 10970
Frame: 10971
Frame: 10972
Frame: 10973
Frame: 10974
Frame: 10975
Frame: 10976
Frame: 10977
Frame: 10978
Frame: 10979
Frame: 10980
Frame: 10981
Frame: 10982
Frame: 10983
Frame: 10984
Frame: 10985
Frame: 10986
Frame: 10987
Frame: 10988
Frame: 10989
Frame: 10990
Frame: 10991
Frame: 10992
Frame: 10993
Frame: 10994
Frame: 10995
Frame: 10996
Frame: 10997
Frame: 10998
Frame: 10999
Frame: 11000
Frame: 11001
Frame: 11002
Frame: 11003
Frame: 11004
Frame: 11005
Frame: 11006
Frame: 11007
Frame: 11008
Frame: 11009
Frame: 11010
Frame: 11011
Frame: 11012
Frame: 11013
Frame: 11014
Frame: 11015
Frame: 11016
Frame: 11017
Frame: 11018
Frame: 11019
Frame: 11020
Frame: 11021
Frame: 11022
Frame: 11023
Frame: 11024
Frame: 11025
Frame: 11026
Frame: 11027
Frame: 11028
Frame: 11029
Frame: 11030
Frame: 11031
Frame: 11032
Frame: 11033
Frame: 11034
Frame: 11035
Frame: 11036
Frame: 11037
Frame: 11038
Frame: 11039

Frame: 11597
Frame: 11598
Frame: 11599
Frame: 11600
Frame: 11601
Frame: 11602
Frame: 11603
Frame: 11604
Frame: 11605
Frame: 11606
Frame: 11607
Frame: 11608
Frame: 11609
Frame: 11610
Frame: 11611
Frame: 11612
Frame: 11613
Frame: 11614
Frame: 11615
Frame: 11616
Frame: 11617
Frame: 11618
Frame: 11619
Frame: 11620
Frame: 11621
Frame: 11622
Frame: 11623
Frame: 11624
Frame: 11625
Frame: 11626
Frame: 11627
Frame: 11628
Frame: 11629
Frame: 11630
Frame: 11631
Frame: 11632
Frame: 11633
Frame: 11634
Frame: 11635
Frame: 11636
Frame: 11637
Frame: 11638
Frame: 11639
Frame: 11640
Frame: 11641
Frame: 11642
Frame: 11643
Frame: 11644
Frame: 11645
Frame: 11646
Frame: 11647
Frame: 11648
Frame: 11649
Frame: 11650
Frame: 11651
Frame: 11652
Frame: 11653
Frame: 11654
Frame: 11655
Frame: 11656
Frame: 11657


In [42]:
### This is the same as the previous, but just pointing at a separate video file. Please
### see the comments in the above cells.
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    videogen = skvideo.io.vreader(PATH_TO_VIDEOS_DIR + "fighter_compilation.mp4")
    writer = skvideo.io.FFmpegWriter(PATH_TO_VIDEOS_DIR + "fighter_compilation_skvid_HQ.mp4")
    
    i=0
    for frame in videogen:
        print("Frame:",i)
        i += 1
        
        frame_expanded = np.expand_dims(frame, axis=0)

        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})

        # Draw the results of the detection (aka 'visulaize the results')
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)
        
        writer.writeFrame(frame)
    writer.close()

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24
Frame: 25
Frame: 26
Frame: 27
Frame: 28
Frame: 29
Frame: 30
Frame: 31
Frame: 32
Frame: 33
Frame: 34
Frame: 35
Frame: 36
Frame: 37
Frame: 38
Frame: 39
Frame: 40
Frame: 41
Frame: 42
Frame: 43
Frame: 44
Frame: 45
Frame: 46
Frame: 47
Frame: 48
Frame: 49
Frame: 50
Frame: 51
Frame: 52
Frame: 53
Frame: 54
Frame: 55
Frame: 56
Frame: 57
Frame: 58
Frame: 59
Frame: 60
Frame: 61
Frame: 62
Frame: 63
Frame: 64
Frame: 65
Frame: 66
Frame: 67
Frame: 68
Frame: 69
Frame: 70
Frame: 71
Frame: 72
Frame: 73
Frame: 74
Frame: 75
Frame: 76
Frame: 77
Frame: 78
Frame: 79
Frame: 80
Frame: 81
Frame: 82
Frame: 83
Frame: 84
Frame: 85
Frame: 86
Frame: 87
Frame: 88
Frame: 89
Frame: 90
Frame: 91
Frame: 92
Frame: 93
Frame: 94
Frame: 95
Frame: 96
Frame: 97
Frame: 98
Frame: 99
Frame: 100

Frame: 756
Frame: 757
Frame: 758
Frame: 759
Frame: 760
Frame: 761
Frame: 762
Frame: 763
Frame: 764
Frame: 765
Frame: 766
Frame: 767
Frame: 768
Frame: 769
Frame: 770
Frame: 771
Frame: 772
Frame: 773
Frame: 774
Frame: 775
Frame: 776
Frame: 777
Frame: 778
Frame: 779
Frame: 780
Frame: 781
Frame: 782
Frame: 783
Frame: 784
Frame: 785
Frame: 786
Frame: 787
Frame: 788
Frame: 789
Frame: 790
Frame: 791
Frame: 792
Frame: 793
Frame: 794
Frame: 795
Frame: 796
Frame: 797
Frame: 798
Frame: 799
Frame: 800
Frame: 801
Frame: 802
Frame: 803
Frame: 804
Frame: 805
Frame: 806
Frame: 807
Frame: 808
Frame: 809
Frame: 810
Frame: 811
Frame: 812
Frame: 813
Frame: 814
Frame: 815
Frame: 816
Frame: 817
Frame: 818
Frame: 819
Frame: 820
Frame: 821
Frame: 822
Frame: 823
Frame: 824
Frame: 825
Frame: 826
Frame: 827
Frame: 828
Frame: 829
Frame: 830
Frame: 831
Frame: 832
Frame: 833
Frame: 834
Frame: 835
Frame: 836
Frame: 837
Frame: 838
Frame: 839
Frame: 840
Frame: 841
Frame: 842
Frame: 843
Frame: 844
Frame: 845
Frame: 846

Frame: 1461
Frame: 1462
Frame: 1463
Frame: 1464
Frame: 1465
Frame: 1466
Frame: 1467
Frame: 1468
Frame: 1469
Frame: 1470
Frame: 1471
Frame: 1472
Frame: 1473
Frame: 1474
Frame: 1475
Frame: 1476
Frame: 1477
Frame: 1478
Frame: 1479
Frame: 1480
Frame: 1481
Frame: 1482
Frame: 1483
Frame: 1484
Frame: 1485
Frame: 1486
Frame: 1487
Frame: 1488
Frame: 1489
Frame: 1490
Frame: 1491
Frame: 1492
Frame: 1493
Frame: 1494
Frame: 1495
Frame: 1496
Frame: 1497
Frame: 1498
Frame: 1499
Frame: 1500
Frame: 1501
Frame: 1502
Frame: 1503
Frame: 1504
Frame: 1505
Frame: 1506
Frame: 1507
Frame: 1508
Frame: 1509
Frame: 1510
Frame: 1511
Frame: 1512
Frame: 1513
Frame: 1514
Frame: 1515
Frame: 1516
Frame: 1517
Frame: 1518
Frame: 1519
Frame: 1520
Frame: 1521
Frame: 1522
Frame: 1523
Frame: 1524
Frame: 1525
Frame: 1526
Frame: 1527
Frame: 1528
Frame: 1529
Frame: 1530
Frame: 1531
Frame: 1532
Frame: 1533
Frame: 1534
Frame: 1535
Frame: 1536
Frame: 1537
Frame: 1538
Frame: 1539
Frame: 1540
Frame: 1541
Frame: 1542
Frame: 1543
Fram

Frame: 2145
Frame: 2146
Frame: 2147
Frame: 2148
Frame: 2149
Frame: 2150
Frame: 2151
Frame: 2152
Frame: 2153
Frame: 2154
Frame: 2155
Frame: 2156
Frame: 2157
Frame: 2158
Frame: 2159
Frame: 2160
Frame: 2161
Frame: 2162
Frame: 2163
Frame: 2164
Frame: 2165
Frame: 2166
Frame: 2167
Frame: 2168
Frame: 2169
Frame: 2170
Frame: 2171
Frame: 2172
Frame: 2173
Frame: 2174
Frame: 2175
Frame: 2176
Frame: 2177
Frame: 2178
Frame: 2179
Frame: 2180
Frame: 2181
Frame: 2182
Frame: 2183
Frame: 2184
Frame: 2185
Frame: 2186
Frame: 2187
Frame: 2188
Frame: 2189
Frame: 2190
Frame: 2191
Frame: 2192
Frame: 2193
Frame: 2194
Frame: 2195
Frame: 2196
Frame: 2197
Frame: 2198
Frame: 2199
Frame: 2200
Frame: 2201
Frame: 2202
Frame: 2203
Frame: 2204
Frame: 2205
Frame: 2206
Frame: 2207
Frame: 2208
Frame: 2209
Frame: 2210
Frame: 2211
Frame: 2212
Frame: 2213
Frame: 2214
Frame: 2215
Frame: 2216
Frame: 2217
Frame: 2218
Frame: 2219
Frame: 2220
Frame: 2221
Frame: 2222
Frame: 2223
Frame: 2224
Frame: 2225
Frame: 2226
Frame: 2227
Fram

Frame: 2830
Frame: 2831
Frame: 2832
Frame: 2833
Frame: 2834
Frame: 2835
Frame: 2836
Frame: 2837
Frame: 2838
Frame: 2839
Frame: 2840
Frame: 2841
Frame: 2842
Frame: 2843
Frame: 2844
Frame: 2845
Frame: 2846
Frame: 2847
Frame: 2848
Frame: 2849
Frame: 2850
Frame: 2851
Frame: 2852
Frame: 2853
Frame: 2854
Frame: 2855
Frame: 2856
Frame: 2857
Frame: 2858
Frame: 2859
Frame: 2860
Frame: 2861
Frame: 2862
Frame: 2863
Frame: 2864
Frame: 2865
Frame: 2866
Frame: 2867
Frame: 2868
Frame: 2869
Frame: 2870
Frame: 2871
Frame: 2872
Frame: 2873
Frame: 2874
Frame: 2875
Frame: 2876
Frame: 2877
Frame: 2878
Frame: 2879
Frame: 2880
Frame: 2881
Frame: 2882
Frame: 2883
Frame: 2884
Frame: 2885
Frame: 2886
Frame: 2887
Frame: 2888
Frame: 2889
Frame: 2890
Frame: 2891
Frame: 2892
Frame: 2893
Frame: 2894
Frame: 2895
Frame: 2896
Frame: 2897
Frame: 2898
Frame: 2899
Frame: 2900
Frame: 2901
Frame: 2902
Frame: 2903
Frame: 2904
Frame: 2905
Frame: 2906
Frame: 2907
Frame: 2908
Frame: 2909
Frame: 2910
Frame: 2911
Frame: 2912
Fram

...

Frame: 4882
Frame: 4883
Frame: 4884
Frame: 4885
Frame: 4886
Frame: 4887
Frame: 4888
Frame: 4889
Frame: 4890
Frame: 4891
Frame: 4892
Frame: 4893
Frame: 4894
Frame: 4895
Frame: 4896
Frame: 4897
Frame: 4898
Frame: 4899
Frame: 4900
Frame: 4901
Frame: 4902
Frame: 4903
Frame: 4904
Frame: 4905
Frame: 4906
Frame: 4907
Frame: 4908
Frame: 4909
Frame: 4910
Frame: 4911
Frame: 4912
Frame: 4913
Frame: 4914
Frame: 4915
Frame: 4916
Frame: 4917
Frame: 4918
Frame: 4919
Frame: 4920
Frame: 4921
Frame: 4922
Frame: 4923
Frame: 4924
Frame: 4925
Frame: 4926
Frame: 4927
Frame: 4928
Frame: 4929
Frame: 4930
Frame: 4931
Frame: 4932
Frame: 4933
Frame: 4934
Frame: 4935
Frame: 4936
Frame: 4937
Frame: 4938
Frame: 4939
Frame: 4940
Frame: 4941
Frame: 4942
Frame: 4943
Frame: 4944
Frame: 4945
Frame: 4946
Frame: 4947
Frame: 4948
Frame: 4949
Frame: 4950
Frame: 4951
Frame: 4952
Frame: 4953
Frame: 4954
Frame: 4955
Frame: 4956
Frame: 4957
Frame: 4958
Frame: 4959
Frame: 4960
Frame: 4961
Frame: 4962
Frame: 4963
Frame: 4964
Fram

Frame: 5567
Frame: 5568
Frame: 5569
Frame: 5570
Frame: 5571
Frame: 5572
Frame: 5573
Frame: 5574
Frame: 5575
Frame: 5576
Frame: 5577
Frame: 5578
Frame: 5579
Frame: 5580
Frame: 5581
Frame: 5582
Frame: 5583
Frame: 5584
Frame: 5585
Frame: 5586
Frame: 5587
Frame: 5588
Frame: 5589
Frame: 5590
Frame: 5591
Frame: 5592
Frame: 5593
Frame: 5594
Frame: 5595
Frame: 5596
Frame: 5597
Frame: 5598
Frame: 5599
Frame: 5600
Frame: 5601
Frame: 5602
Frame: 5603
Frame: 5604
Frame: 5605
Frame: 5606
Frame: 5607
Frame: 5608
Frame: 5609
Frame: 5610
Frame: 5611
Frame: 5612
Frame: 5613
Frame: 5614
Frame: 5615
Frame: 5616
Frame: 5617
Frame: 5618
Frame: 5619
Frame: 5620
Frame: 5621
Frame: 5622
Frame: 5623
Frame: 5624
Frame: 5625
Frame: 5626
Frame: 5627
Frame: 5628
Frame: 5629
Frame: 5630
Frame: 5631
Frame: 5632
Frame: 5633
Frame: 5634
Frame: 5635
Frame: 5636
Frame: 5637
Frame: 5638
Frame: 5639
Frame: 5640
Frame: 5641
Frame: 5642
Frame: 5643
Frame: 5644
Frame: 5645
Frame: 5646
Frame: 5647
Frame: 5648
Frame: 5649
Fram

Frame: 6250
Frame: 6251
Frame: 6252
Frame: 6253
Frame: 6254
Frame: 6255
Frame: 6256
Frame: 6257
Frame: 6258
Frame: 6259
Frame: 6260
Frame: 6261
Frame: 6262
Frame: 6263
Frame: 6264
Frame: 6265
Frame: 6266
Frame: 6267
Frame: 6268
Frame: 6269
Frame: 6270
Frame: 6271
Frame: 6272
Frame: 6273
Frame: 6274
Frame: 6275
Frame: 6276
Frame: 6277
Frame: 6278
Frame: 6279
Frame: 6280
Frame: 6281
Frame: 6282
Frame: 6283
Frame: 6284
Frame: 6285
Frame: 6286
Frame: 6287
Frame: 6288
Frame: 6289
Frame: 6290
Frame: 6291
Frame: 6292
Frame: 6293
Frame: 6294
Frame: 6295
Frame: 6296
Frame: 6297
Frame: 6298
Frame: 6299
Frame: 6300
Frame: 6301
Frame: 6302
Frame: 6303
Frame: 6304
Frame: 6305
Frame: 6306
Frame: 6307
Frame: 6308
Frame: 6309
Frame: 6310
Frame: 6311
Frame: 6312
Frame: 6313
Frame: 6314
Frame: 6315
Frame: 6316
Frame: 6317
Frame: 6318
Frame: 6319
Frame: 6320
Frame: 6321
Frame: 6322
Frame: 6323
Frame: 6324
Frame: 6325
Frame: 6326
Frame: 6327
Frame: 6328
Frame: 6329
Frame: 6330
Frame: 6331
Frame: 6332
Fram

Frame: 6934
Frame: 6935
Frame: 6936
Frame: 6937
Frame: 6938
Frame: 6939
Frame: 6940
Frame: 6941
Frame: 6942
Frame: 6943
Frame: 6944
Frame: 6945
Frame: 6946
Frame: 6947
Frame: 6948
Frame: 6949
Frame: 6950
Frame: 6951
Frame: 6952
Frame: 6953
Frame: 6954
Frame: 6955
Frame: 6956
Frame: 6957
Frame: 6958
Frame: 6959
Frame: 6960
Frame: 6961
Frame: 6962
Frame: 6963
Frame: 6964
Frame: 6965
Frame: 6966
Frame: 6967
Frame: 6968
Frame: 6969
Frame: 6970
Frame: 6971
Frame: 6972
Frame: 6973
Frame: 6974
Frame: 6975
Frame: 6976
Frame: 6977
Frame: 6978
Frame: 6979
Frame: 6980
Frame: 6981
Frame: 6982
Frame: 6983
Frame: 6984
Frame: 6985
Frame: 6986
Frame: 6987
Frame: 6988
Frame: 6989
Frame: 6990
Frame: 6991
Frame: 6992
Frame: 6993
Frame: 6994
Frame: 6995
Frame: 6996
Frame: 6997
Frame: 6998
Frame: 6999
Frame: 7000
Frame: 7001
Frame: 7002
Frame: 7003
Frame: 7004
Frame: 7005
Frame: 7006
Frame: 7007
Frame: 7008
Frame: 7009
Frame: 7010
Frame: 7011
Frame: 7012
Frame: 7013
Frame: 7014
Frame: 7015
Frame: 7016
Fram

Frame: 7618
Frame: 7619
Frame: 7620
Frame: 7621
Frame: 7622
Frame: 7623
Frame: 7624
Frame: 7625
Frame: 7626
Frame: 7627
Frame: 7628
Frame: 7629
Frame: 7630
Frame: 7631
Frame: 7632
Frame: 7633
Frame: 7634
Frame: 7635
Frame: 7636
Frame: 7637
Frame: 7638
Frame: 7639
Frame: 7640
Frame: 7641
Frame: 7642
Frame: 7643
Frame: 7644
Frame: 7645
Frame: 7646
Frame: 7647
Frame: 7648
Frame: 7649
Frame: 7650
Frame: 7651
Frame: 7652
Frame: 7653
Frame: 7654
Frame: 7655
Frame: 7656
Frame: 7657
Frame: 7658
Frame: 7659
Frame: 7660
Frame: 7661
Frame: 7662
Frame: 7663
Frame: 7664
Frame: 7665
Frame: 7666
Frame: 7667
Frame: 7668
Frame: 7669
Frame: 7670
Frame: 7671
Frame: 7672
Frame: 7673
Frame: 7674
Frame: 7675
Frame: 7676
Frame: 7677
Frame: 7678
Frame: 7679
Frame: 7680
Frame: 7681
Frame: 7682
Frame: 7683
Frame: 7684
Frame: 7685
Frame: 7686
Frame: 7687
Frame: 7688
Frame: 7689
Frame: 7690
Frame: 7691
Frame: 7692
Frame: 7693
Frame: 7694
Frame: 7695
Frame: 7696
Frame: 7697
Frame: 7698
Frame: 7699
Frame: 7700
Fram